In [11]:
import pandas as pd

In [12]:
#Carrega o arquivo CSV com os dados e coordenadas
planilha = pd.read_excel('bd.xlsx')  # Substitua pelo caminho da database com cordenadas
display(planilha)

,Bairro,módulo,Latitude/Longitude,Latitude,Longitude,Quantidade
0,ALTO MARON,Módulo 3,"-14.782406201236284, -39.27104257565622",-14.782406,-39.271043,11
1,ANTIQUE,Módulo 1,"-14.7727307312968, -39.27436689867163",-14.772731,-39.274367,7
2,BANANEIRA,Módulo 2,"-14.810091999686808, -39.28894134721365",-14.810092,-39.288941,2
3,BANCO RASO,Módulo 4,"-14.805182633594178, -39.28096296374052",-14.805183,-39.280963,9
4,BERILO,Módulo 3,"-14.799426635325863, -39.279588979911004",-14.799427,-39.279589,28
...,...,...,...,...,...,...
64,VILA ANÁLIA,Módulo 4,"-14.818996888662442, -39.27650466162148",-14.818997,-39.276505,27
65,VILA DAS DORES,Módulo 1,"-14.784733082917056, -39.26005677355378",-14.784733,-39.260057,28
66,VILA ZARA,Módulo 4,"-14.786107416759247, -39.26650959961409",-14.786107,-39.266510,15
67,ZILDOLANDIA,Módulo 3,"-14.796040921973935, -39.279951539872755",-14.796041,-39.279952,18


In [26]:
import plotly.express as px
grafico = px.density_mapbox(planilha, lat='Latitude', lon='Longitude', z='Quantidade', mapbox_style="open-street-map", radius=20, zoom=11)
grafico.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
grafico.show()